In [1]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import json
import io

print("🚀 Starting Fragile Watermarking System...")

🚀 Starting Fragile Watermarking System...


In [2]:
# 1. Core Functions (Same as before)
def load_and_preprocess_image(image_path):
    """Load and preprocess image for watermarking"""
    try:
        img = Image.open(image_path)
        img_array = np.array(img)
        print(f"✅ Image loaded: {img_array.shape}")
        return img_array, 'color' if len(img_array.shape) == 3 else 'grayscale'
    except Exception as e:
        print(f"❌ Error loading image: {e}")
        return None, None

def text_to_binary(text):
    """Convert text to binary"""
    return ''.join(format(ord(char), '08b') for char in text)

def binary_to_text(binary_str):
    """Convert binary back to text"""
    text = ''
    for i in range(0, len(binary_str), 8):
        byte = binary_str[i:i+8]
        if len(byte) == 8:
            text += chr(int(byte, 2))
    return text

def embed_watermark_lsb(image_array, watermark_text, mode='color'):
    """Embed watermark using LSB"""
    binary_watermark = text_to_binary(watermark_text) + '00000000'
    watermark_length = len(binary_watermark)

    watermarked_image = image_array.copy()

    if mode == 'color':
        flat_pixels = watermarked_image.reshape(-1)
    else:
        flat_pixels = watermarked_image.reshape(-1)

    if watermark_length > len(flat_pixels):
        raise ValueError("❌ Watermark too large for image!")

    # Embed the watermark
    for i in range(watermark_length):
        flat_pixels[i] = (flat_pixels[i] & 0xFE) | int(binary_watermark[i])

    if mode == 'color':
        watermarked_image = flat_pixels.reshape(image_array.shape)
    else:
        watermarked_image = flat_pixels.reshape(image_array.shape)

    return watermarked_image, watermark_length

def extract_watermark_lsb(image_array, watermark_length, mode='color'):
    """Extract watermark from LSB"""
    if mode == 'color':
        flat_pixels = image_array.reshape(-1)
    else:
        flat_pixels = image_array.reshape(-1)

    binary_watermark = ''.join(str(flat_pixels[i] & 1) for i in range(watermark_length))
    return binary_to_text(binary_watermark)

def calculate_quality_metrics(original, watermarked):
    """Calculate image quality metrics"""
    mse = np.mean((original.astype(float) - watermarked.astype(float)) ** 2)
    psnr = 20 * np.log10(255.0 / np.sqrt(mse)) if mse > 0 else float('inf')
    return {'MSE': mse, 'PSNR': psnr}

def save_watermark_info(watermark_text, watermark_length, output_path):
    """Save watermark information to a JSON file for verification"""
    info = {
        'watermark_text': watermark_text,
        'watermark_length': watermark_length,
        'timestamp': np.datetime64('now').astype(str)
    }

    info_path = output_path.replace('.png', '_watermark_info.json')
    with open(info_path, 'w') as f:
        json.dump(info, f, indent=2)

    return info_path

def load_watermark_info(info_path):
    """Load watermark information from JSON file"""
    try:
        with open(info_path, 'r') as f:
            return json.load(f)
    except:
        return None



In [3]:
# Add these functions after your existing core functions

def calculate_tamper_percentage(expected, extracted):
    """Calculate comprehensive tamper analysis metrics"""
    
    # Remove null characters and strip
    extracted_clean = extracted.replace(chr(0), '').strip()
    expected_clean = expected.strip()
    
    # Character-level comparison
    min_len = min(len(expected_clean), len(extracted_clean))
    max_len = max(len(expected_clean), len(extracted_clean))
    
    if min_len == 0:
        return {
            'tamper_percentage': 100.0,
            'character_match': 0.0,
            'word_match': 0.0,
            'sequence_similarity': 0.0,
            'length_similarity': 0.0,
            'overall_confidence': 0.0
        }
    
    # Character match percentage
    char_matches = sum(1 for i in range(min_len) if expected_clean[i] == extracted_clean[i])
    character_match = (char_matches / min_len) * 100
    
    # Word-level comparison
    expected_words = expected_clean.split()
    extracted_words = extracted_clean.split()
    min_words = min(len(expected_words), len(extracted_words))
    
    if min_words > 0:
        word_matches = sum(1 for i in range(min_words) if expected_words[i] == extracted_words[i])
        word_match = (word_matches / min_words) * 100
    else:
        word_match = 0.0
    
    # Sequence similarity (how much of expected appears in extracted)
    sequence_similarity = 0
    for i in range(len(expected_clean)):
        for j in range(i + 1, len(expected_clean) + 1):
            substring = expected_clean[i:j]
            if substring in extracted_clean and len(substring) > 1:
                sequence_similarity = max(sequence_similarity, len(substring))
    
    sequence_similarity = (sequence_similarity / len(expected_clean)) * 100 if expected_clean else 0
    
    # Length similarity
    length_similarity = (min_len / max_len) * 100 if max_len > 0 else 0
    
    # Overall tamper percentage (weighted average)
    tamper_percentage = 100 - (
        0.4 * character_match +
        0.3 * word_match +
        0.2 * sequence_similarity +
        0.1 * length_similarity
    )
    
    # Overall confidence (inverse of tamper percentage)
    overall_confidence = max(0, 100 - tamper_percentage)
    
    return {
        'tamper_percentage': tamper_percentage,
        'character_match': character_match,
        'word_match': word_match,
        'sequence_similarity': sequence_similarity,
        'length_similarity': length_similarity,
        'overall_confidence': overall_confidence
    }

def get_tamper_severity(confidence):
    """Determine tamper severity based on confidence level"""
    if confidence >= 95:
        return "EXCELLENT", "Image appears completely authentic"
    elif confidence >= 90:
        return "VERY GOOD", "Minor, likely insignificant alterations"
    elif confidence >= 80:
        return "GOOD", "Slight modifications detected"
    elif confidence >= 70:
        return "MODERATE", "Noticeable alterations present"
    elif confidence >= 60:
        return "SUSPICIOUS", "Significant tampering likely"
    elif confidence >= 50:
        return "POOR", "Heavy tampering suspected"
    elif confidence >= 30:
        return "VERY POOR", "Extensive modification confirmed"
    else:
        return "REJECTED", "Image has been completely altered"

def analyze_tamper_pattern(expected, extracted):
    """Analyze and display specific tamper patterns"""
    expected_clean = expected.strip()
    extracted_clean = extracted.replace(chr(0), '').strip()
    
    print(f"\n🔬 DETAILED TAMPER ANALYSIS:")
    
    # Length analysis
    if len(extracted_clean) != len(expected_clean):
        print(f" Length mismatch: Expected {len(expected_clean)}, Got {len(extracted_clean)}")
    
    # Character-by-character analysis for short strings
    if len(expected_clean) <= 50:
        print(f"   • Character comparison:")
        for i in range(min(len(expected_clean), len(extracted_clean))):
            if expected_clean[i] != extracted_clean[i]:
                print(f"     Position {i+1}: Expected '{expected_clean[i]}', Got '{extracted_clean[i]}'")
    
    # Check for common tamper patterns
    if chr(0) in extracted:
        null_count = extracted.count(chr(0))
        print(f"   • Found {null_count} null characters (common in LSB corruption)")
    
    # Bit error analysis
    expected_binary = text_to_binary(expected_clean)
    extracted_binary = text_to_binary(extracted_clean)
    
    min_bits = min(len(expected_binary), len(extracted_binary))
    if min_bits > 0:
        bit_errors = sum(1 for i in range(min_bits) if expected_binary[i] != extracted_binary[i])
        bit_error_rate = (bit_errors / min_bits) * 100
        print(f"   • Bit error rate: {bit_error_rate:.1f}% ({bit_errors}/{min_bits} bits)")

In [4]:
# 2. BULLETPROOF UPLOAD HANDLER
def handle_uploaded_file(upload_widget):
    """
    Ultra-robust upload handler that handles ALL ipywidgets formats
    """
    if not upload_widget.value:
        print("❌ No upload widget value")
        return None, None

    try:
        upload_data = upload_widget.value
        print(f"🔍 Upload data type: {type(upload_data)}")
        print(f"🔍 Upload data: {upload_data}")

        # Method 1: Direct content access (simplest approach)
        if hasattr(upload_data, 'content'):
            print("✅ Using direct content access")
            return upload_data.content, "uploaded_file.png"

        # Method 2: Dictionary format (common in newer ipywidgets)
        if isinstance(upload_data, dict):
            print("📦 Processing as dictionary...")
            # Try different dictionary structures
            for key, value in upload_data.items():
                print(f"   Key: {key}, Type: {type(value)}")
                if isinstance(value, dict):
                    if 'content' in value:
                        content = value['content']
                        filename = value.get('name', 'uploaded_file.png')
                        print(f"✅ Found content in nested dict: {filename}")
                        return content, filename
                elif hasattr(value, 'content'):
                    content = value.content
                    filename = getattr(value, 'name', 'uploaded_file.png')
                    print(f"✅ Found content in object: {filename}")
                    return content, filename

            # If we get here, try to extract first value
            first_value = list(upload_data.values())[0]
            if hasattr(first_value, 'content'):
                content = first_value.content
                filename = getattr(first_value, 'name', 'uploaded_file.png')
                print(f"✅ Using first value content: {filename}")
                return content, filename

        # Method 3: List/Tuple format
        if isinstance(upload_data, (list, tuple)) and len(upload_data) > 0:
            print("📋 Processing as list/tuple...")
            first_item = upload_data[0]
            print(f"   First item type: {type(first_item)}")

            if isinstance(first_item, dict):
                if 'content' in first_item:
                    content = first_item['content']
                    filename = first_item.get('name', 'uploaded_file.png')
                    print(f"✅ Found content in list dict: {filename}")
                    return content, filename
            elif hasattr(first_item, 'content'):
                content = first_item.content
                filename = getattr(first_item, 'name', 'uploaded_file.png')
                print(f"✅ Found content in list object: {filename}")
                return content, filename

        # Method 4: Last resort - try to iterate through everything
        print("🔄 Trying deep search...")
        def deep_search(obj, path=""):
            if hasattr(obj, 'content'):
                print(f"✅ Found content at {path}")
                return obj.content, getattr(obj, 'name', 'uploaded_file.png')
            elif isinstance(obj, dict):
                for k, v in obj.items():
                    result = deep_search(v, f"{path}.{k}")
                    if result[0] is not None:
                        return result
            elif isinstance(obj, (list, tuple)):
                for i, item in enumerate(obj):
                    result = deep_search(item, f"{path}[{i}]")
                    if result[0] is not None:
                        return result
            return None, None

        content, filename = deep_search(upload_data, "root")
        if content is not None:
            return content, filename

        print("❌ Could not find content in any format")
        return None, None

    except Exception as e:
        print(f"❌ Upload handling error: {e}")
        import traceback
        print(f"🔍 Detailed error: {traceback.format_exc()}")
        return None, None

In [5]:
# 3. SIMPLIFIED EMBEDDING INTERFACE
def create_embedding_interface():
    """Simplified embedding interface"""

    upload = widgets.FileUpload(
        description='📁 Choose Image File',
        accept='.jpg,.jpeg,.png,.bmp,.tiff',
        multiple=False
    )

    watermark_text = widgets.Text(
        value='© Copyright 2024',
        description='Watermark:',
        layout=widgets.Layout(width='400px')
    )

    embed_btn = widgets.Button(
        description='🔒 Embed Watermark',
        button_style='success',
        layout=widgets.Layout(width='200px', height='40px')
    )

    output_area = widgets.Output()

    def embed_watermark(b):
        with output_area:
            clear_output()
            print("🔄 Starting embedding process...")

            # Get uploaded file
            content, filename = handle_uploaded_file(upload)
            if content is None:
                print("❌ Please select an image file first!")
                return

            print(f"✅ File selected: {filename}")

            if not watermark_text.value.strip():
                print("❌ Please enter watermark text!")
                return

            try:
                # Create temporary file
                temp_path = f"temp_{np.random.randint(10000, 99999)}.png"
                with open(temp_path, 'wb') as f:
                    f.write(content)

                print("📥 Loading image...")
                original_image, image_mode = load_and_preprocess_image(temp_path)
                if original_image is None:
                    return

                print(f"💧 Embedding: '{watermark_text.value}'")
                watermarked, wm_length = embed_watermark_lsb(
                    original_image, watermark_text.value, image_mode
                )

                # Display results
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

                if image_mode == 'color':
                    ax1.imshow(original_image)
                    ax2.imshow(watermarked)
                else:
                    ax1.imshow(original_image, cmap='gray')
                    ax2.imshow(watermarked, cmap='gray')

                ax1.set_title('Original Image')
                ax2.set_title('Watermarked Image')
                ax1.axis('off')
                ax2.axis('off')
                plt.tight_layout()
                plt.show()

                # Save files
                output_path = 'watermarked_image.png'
                Image.fromarray(watermarked).save(output_path)
                info_path = save_watermark_info(watermark_text.value, wm_length, output_path)

                print(f"✅ Embedding successful!")
                print(f"📁 Watermarked image: {output_path}")
                print(f"📄 Watermark info: {info_path}")

            except Exception as e:
                print(f"❌ Error: {e}")
            finally:
                if os.path.exists(temp_path):
                    os.remove(temp_path)

    embed_btn.on_click(embed_watermark)

    return widgets.VBox([
        widgets.HTML("<h3>🔒 Embed Watermark</h3>"),
        upload,
        watermark_text,
        embed_btn,
        output_area
    ])

In [6]:
# 4. SIMPLIFIED VERIFICATION INTERFACE
def create_verification_interface():
    """Simplified verification interface"""

    image_upload = widgets.FileUpload(
        description='📁 Image to Verify',
        accept='.jpg,.jpeg,.png,.bmp,.tiff',
        multiple=False
    )

    expected_text = widgets.Text(
        value='',
        description='Expected text:',
        placeholder='Enter the watermark text you expect...',
        layout=widgets.Layout(width='400px')
    )

    verify_btn = widgets.Button(
        description='🔍 Verify',
        button_style='info',
        layout=widgets.Layout(width='200px', height='40px')
    )

    output_area = widgets.Output()

    def verify_watermark(b):
        with output_area:
            clear_output()
            print("🔄 Starting verification...")

            content, filename = handle_uploaded_file(image_upload)
            if content is None:
                print("❌ Please select an image file!")
                return

            if not expected_text.value.strip():
                print("❌ Please enter expected watermark text!")
                return

            try:
                # Save and load image
                temp_path = f"verify_{np.random.randint(10000, 99999)}.png"
                with open(temp_path, 'wb') as f:
                    f.write(content)
                
                image, mode = load_and_preprocess_image(temp_path)
                if image is None:
                    return
                
                # Calculate watermark length and extract
                wm_length = len(text_to_binary(expected_text.value)) + 8
                extracted = extract_watermark_lsb(image, wm_length, mode)
                
                # Calculate tamper percentages
                metrics = calculate_tamper_percentage(expected_text.value, extracted)
                
                # Display comprehensive results
                print("\n" + "="*60)
                print("📊 COMPREHENSIVE TAMPER ANALYSIS REPORT")
                print("="*60)
                
                # Overall verdict
                tamper_pct = metrics['tamper_percentage']
                severity, message = get_tamper_severity(metrics['overall_confidence'])
                
                print(f"\n🎯 OVERALL VERDICT: {severity} - {message}")
                print(f"📈 AUTHENTICITY CONFIDENCE: {metrics['overall_confidence']}%")
                print(f"📉 TAMPER PERCENTAGE: {tamper_pct}%")
                
                # Detailed metrics
                print(f"\n📋 DETAILED METRICS:")
                print(f"   • Character Match: {metrics['character_match']}%")
                print(f"   • Word Match: {metrics['word_match']}%")
                print(f"   • Sequence Similarity: {metrics['sequence_similarity']}%")
                print(f"   • Length Similarity: {metrics['length_similarity']}%")
                
                # Watermark comparison
                print(f"\n🔍 WATERMARK COMPARISON:")
                print(f"   Expected: '{expected_text.value}'")
                print(f"   Extracted: '{extracted.replace(chr(0), '').strip()}'")
                
                # Detailed tamper analysis
                analyze_tamper_pattern(expected_text.value, extracted)
                
                # Visual representation
                print(f"\n📊 VISUAL TAMPER SCALE:")
                confidence = metrics['overall_confidence']
                bars = int(confidence / 5)  # 20 bars total
                tamper_bars = 20 - bars
                
                print("   Authentic " + "█" * bars + " " * tamper_bars + " Tampered")
                print(f"   {confidence:5.1f}%{' ' * 12}{100-confidence:5.1f}%")
                
                # Risk assessment
                print(f"\n⚠️  RISK ASSESSMENT:")
                if confidence >= 90:
                    print("   ✅ LOW RISK - Image appears authentic")
                elif confidence >= 70:
                    print("   ⚠️  MEDIUM RISK - Minor alterations detected")
                elif confidence >= 50:
                    print("   🔥 HIGH RISK - Significant tampering suspected")
                else:
                    print("   💀 CRITICAL RISK - Heavy tampering confirmed")
                
                # Display the image
                plt.figure(figsize=(10, 8))
                if mode == 'color':
                    plt.imshow(image)
                else:
                    plt.imshow(image, cmap='gray')
                
                # Color code title based on confidence
                title_color = 'green' if confidence >= 90 else 'orange' if confidence >= 70 else 'red'
                plt.title(f'Tamper Analysis: {confidence:.1f}% Confidence\n{message}', 
                         color=title_color, fontsize=14, fontweight='bold')

                plt.axis('off')
                plt.tight_layout()
                plt.show()
                
                # Recommendation
                print(f"\n💡 RECOMMENDATION:")
                if confidence >= 90:
                    print("   This image can be trusted for most purposes")
                elif confidence >= 70:
                    print("   Use with caution - verify through other means if possible")
                elif confidence >= 50:
                    print("   Do not trust this image without additional verification")
                else:
                    print("   REJECT this image - it has been significantly altered")
                
            except Exception as e:
                print(f"❌ Analysis error: {e}")
                import traceback
                print(f"🔍 Detailed error: {traceback.format_exc()}")
            finally:
                if os.path.exists(temp_path):
                    os.remove(temp_path)
    
    verify_btn.on_click(verify_watermark)

    return widgets.VBox([
        widgets.HTML("<h3>🔬 Advanced Tamper Analysis</h3>"),
        widgets.HTML("<p>Upload a watermarked image and expected text to analyze tampering</p>"),
        image_upload,
        expected_text,
        verify_btn,
        output_area
    ])

In [7]:
# 5. Main Interface
def create_main_interface():
    """Create the main interface with tabs"""

    # Create tabs
    embedding_tab = create_embedding_interface()
    verification_tab = create_verification_interface()

    # Create tab container
    tab = widgets.Tab()
    tab.children = [embedding_tab, verification_tab]
    tab.set_title(0, '🔒 EMBED')
    tab.set_title(1, '🔍 VERIFY')

    # Display everything
    display(widgets.VBox([
        widgets.HTML("""
            <h1 style="text-align: center; color: #2E86AB;">🔒 Fragile Watermarking System</h1>
            <p style="text-align: center; color: #666;">Separate Embedding and Verification Workflow</p>
            <hr>
        """),
        tab
    ]))

In [8]:
# 6. ALTERNATIVE: GOOGLE COLAB NATIVE UPLOAD
def colab_native_interface():
    """Use Google Colab's native file upload"""
    from google.colab import files
    import io

    print("📱 Using Google Colab Native Upload")

    def native_embed():
        print("=== EMBED WATERMARK ===")
        uploaded = files.upload()
        if not uploaded:
            print("No file selected!")
            return

        filename = list(uploaded.keys())[0]
        watermark = input("Enter watermark text (or press Enter for default): ").strip()
        if not watermark:
            watermark = "© Copyright 2024"

        # Process image
        image = Image.open(io.BytesIO(uploaded[filename]))
        img_array = np.array(image)
        mode = 'color' if len(img_array.shape) == 3 else 'grayscale'

        print(f"Embedding in {mode} image...")
        watermarked, wm_length = embed_watermark_lsb(img_array, watermark, mode)

        # Save and download
        output_path = 'watermarked_image.png'
        Image.fromarray(watermarked).save(output_path)
        files.download(output_path)

        print(f"✅ Done! Watermark length: {wm_length}")
        print(f"💡 Save this length for verification: {wm_length}")

    def native_verify():
        print("=== VERIFY WATERMARK ===")
        uploaded = files.upload()
        if not uploaded:
            print("No file selected!")
            return

        filename = list(uploaded.keys())[0]
        expected = input("Expected watermark text: ").strip()
        if not expected:
            print("Please enter expected watermark!")
            return

        # Calculate or get length
        use_calculated = input("Use calculated length? (y/n): ").lower() == 'y'
        if use_calculated:
            wm_length = len(text_to_binary(expected)) + 8
        else:
            wm_length = int(input("Enter watermark length: "))

        # Process image
        image = Image.open(io.BytesIO(uploaded[filename]))
        img_array = np.array(image)
        mode = 'color' if len(img_array.shape) == 3 else 'grayscale'

        extracted = extract_watermark_lsb(img_array, wm_length, mode)
        print(f"\n🔍 Results:")
        print(f"Expected: '{expected}'")
        print(f"Found: '{extracted}'")
        print(f"Match: {expected in extracted}")

    print("Choose:")
    print("1. Embed Watermark")
    print("2. Verify Watermark")
    choice = input("Enter 1 or 2: ")

    if choice == '1':
        native_embed()
    elif choice == '2':
        native_verify()
    else:
        print("Invalid choice!")

In [9]:
print("🚀 Initializing...")
try:
    # Try the widget interface first
    create_main_interface()
    print("✅ Widget interface loaded successfully!")
except Exception as e:
    print(f"⚠️ Widget interface failed: {e}")
    print("🔄 Falling back to Colab native interface...")
    colab_native_interface()

🚀 Initializing...


✅ Widget interface loaded successfully!
